<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/debugging_mal.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/debugging_mal.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Debugging MAL
* When MAL Uploads aren't working you can follow some of the steps in this guide

#### 1: Make sure that model assisted labeling is turned on for your project.
    * This is the default, but can be manually turned on as well.
    * You can do that from the web ui or with the following code:
``` project.enable_model_assisted_labeling()```

#### 2: Make sure your import objects or import file matches the following structure:
    * https://docs.labelbox.com/data-model/en/index-en#annotations

#### 3: Make sure your import objects or import file matches the following:
    * Make sure validate is set to true (it is by default) on your upload (Make sure you are using sdk version > 2.5.1)

    

#### Reading validation Errors:
* We get an mal validation error:
* ```MALValidationError: Invalid NDJson on line 0('Invalid NDJson on line 0', None)```
* This means that line 0 (the first example), was invalid.
---
* We can see why it was invalid up above the MALValidationError. 
* The origianal error was a ValuError because we are attempting to attach an example to a data row that is not in the project
--- 
* Errors will be thrown for incorrect or missing keys
* invalid datarow or schema ids
* invalid data types


In [1]:
!pip install labelbox
!pip install ndjson
!pip install requests

In [2]:
# Note that the following code (_validate_ndjson) is run when you perform bulk uploads.
# You don't need to use it directly. This section will show you how to read the error
from labelbox.schema.bulk_import_request import _validate_ndjson
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend
import ndjson
import requests
import uuid
from getpass import getpass
import os

In [5]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

In [6]:
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.POLYGON, name="person"),
])

In [7]:
project = client.create_project(name="debugging_mal_project")
dataset = client.create_dataset(name="debugging_mal_dataset")
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
project.setup(editor, ontology_builder.asdict())
project.datasets.connect(dataset)
ontology = ontology_builder.from_project(project)

In [8]:
list(list(project.datasets())[0].data_rows())[0].uid

'cky4mzhmknvos0zcwexjj9nc0'

In [9]:
poly_example = {
    "uuid":
        "1b5762e9-416c-44cf-9a5f-07effb51f863",
    "schemaId":  #         ontology.tools[0].feature_schema_id,
        list(list(project.datasets())[0].data_rows())[0].uid,
    "dataRow": {
        "id":
            "cky4mzhmknvos0zcwexjj9nc0"  #Invalid (not attached to this project) data_row.uid
    },
    "polygon": [{
        "x": 2,
        "y": 99
    }, {
        "x": 93,
        "y": 5
    }, {
        "x": 51,
        "y": 106
    }, {
        "x": 176,
        "y": 142
    }]
}

In [10]:
#we can see that we did not pass in a valid schemaId - instead, we passed in the dataRow uid

_validate_ndjson([poly_example], project)

MALValidationError: Invalid NDJson on line 0('Invalid NDJson on line 0', None)

#### Reading MAL Errors
* If your upload passes client side validation checks, there is still a chance of server side failures.
* There are two urls for checking the status of your upload
    * bulk_import_request.status_file_url
    * bulk_import_request.error_file_url

In [11]:
invalid_example = poly_example.copy()
invalid_example["polygon"] = [{"x": 2, "y": 99}]

# Old way:
name = str(uuid.uuid4())
upload_task = project.upload_annotations(
    name=name,
    annotations=[poly_example, invalid_example],
    validate=False  # Turn this off or it will raise the error on the client side
)
upload_task.wait_until_done()

In [12]:
# Let's check the status:
print(upload_task.errors)

[{'uuid': '1b5762e9-416c-44cf-9a5f-07effb51f863', 'dataRow': {'id': 'cjxav4aa07r1g0dsq70t9eveg'}, 'status': 'FAILURE', 'errors': [{'name': 'DataRowNotFound', 'message': 'dataRow.id cjxav4aa07r1g0dsq70t9eveg invalid', 'additionalInfo': None}]}, {'uuid': '1b5762e9-416c-44cf-9a5f-07effb51f863', 'dataRow': {'id': 'cjxav4aa07r1g0dsq70t9eveg'}, 'status': 'FAILURE', 'errors': [{'name': 'DataRowNotFound', 'message': 'dataRow.id cjxav4aa07r1g0dsq70t9eveg invalid', 'additionalInfo': None}]}]


In [13]:
# We can see that we are using invalid datarows (they don't belong to the project we are uploading to)
# and that the geometry is invalid.